In [24]:

import pandas as pd
import requests
import altair as alt

collections = ["undervalued_growth_stocks",
               "growth_technology_stocks",
               "undervalued_large_caps",
               "aggressive_small_caps",
               "small_cap_gainers"]

# Lists to store dfs for PE Ratio and Book Value
pe_dfs = []
bv_dfs = []

# List to store box plots
box_plots = []

for collection in collections:
    startIndex = None
    if (collection == "undervalued_large_caps"):
        startIndex = 1
    else:
        startIndex = 0
    # demo api key
    url = f"https://mboum.com/api/v1/co/collections/?list={collection}&start={startIndex}&apikey=demo"
    r = requests.get(url)
    data = r.json()
    quotes = data['data']['quotes']

    # Lists to store data
    symbols = []
    pe_ratios = []
    price_to_book_ratios = []

    for quote in quotes:
        symbols.append(quote['symbol'])
        pe_ratios.append(quote.get('forwardPE', None))
        price_to_book_ratios.append(quote.get('priceToBook', None))

    # df for the current collection
    pe_df = pd.DataFrame({
        'Collection': [collection.replace("_", " ").title()] * len(symbols),
        'Symbol': symbols,
        'PE Ratio': pe_ratios
    })

    bv_df = pd.DataFrame({
        'Collection': [collection.replace("_", " ").title()] * len(symbols),
        'Symbol': symbols,
        'Price to Book': price_to_book_ratios
    })

    pe_dfs.append(pe_df)
    bv_dfs.append(bv_df)

    # combine all dfs into a single one
    combined_pe_df = pd.concat(pe_dfs)
    combined_bv_df = pd.concat(bv_dfs)

    # color schemes
    color_scheme = alt.Scale(range=["#238b45", "#41ab5d", "#78c679", "#addd8e", "#d9f0a3"])


    # Create a single box plot for all PE Ratios
    pe_box_plot = alt.Chart(combined_pe_df).mark_boxplot().encode(
        x=alt.X('Collection:N', title=None),
        y=alt.Y('PE Ratio:Q', axis=alt.Axis(title='PE Ratio')),
        color=alt.Color('Collection:N', legend=None, scale=color_scheme) 
    ).properties(
        width=500, 
        height=500,
        title='PE Ratio Across Collections' 
        )
    
    # Create a single box plot for all Book Values
    bv_box_plot = alt.Chart(combined_bv_df).mark_boxplot().encode(
        x=alt.X('Collection:N', title=None), 
        y=alt.Y('Price to Book:Q', axis=alt.Axis(title='Price to Book'), scale=alt.Scale(domain=[-20, 70])),  
        color=alt.Color('Collection:N', legend=None, scale=color_scheme) 
    ).properties(
        width=500,
        height=500,  
        title='Price to Book Across Collections'
        )


# Combine box plots into a single chart
combined_plots = pe_box_plot | bv_box_plot

combined_plots = combined_plots.configure_title(
    fontSize=16,
    fontWeight='bold',
    color='#006400' 
).configure_axis(
    labelFontSize=12, 
    titleFontSize=14, 
    titleFontWeight='normal', 
)

combined_plots
 


alt.HConcatChart(...)